# 2nd Level Model Structure: Condenser Stage

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.display_utility import trans_product_mole, trans_product_mass

# stage construction rules
from physics.energy_condenser import energy_block_rule
from physics.VLLE import VLLE_block_rule

# collect variable bounds
from physics.bounds import water_x, water_yp

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet
model.COMP_WATER = pe.Set(initialize=['H2O'])

# Global Variables

In [4]:
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(20+273.15,40+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar
model.Q_main = pe.Var(within=pe.Reals) # MW
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.W = pe.Var(within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)

model.H_F = pe.Var(within=pe.Reals)
model.f_V = pe.Var(m.COMP_TOTAL,within=pe.PositiveReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.PositiveReals,initialize=1e-20)

# Construct Individual Blocks

In [5]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [6]:
model.VLE_block = pe.Block(rule=VLLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [7]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    elif i in model.COMP_WATER:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) - model.W == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [8]:
# Equilibrium
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL-model.COMP_WATER,rule=VL_equil_rule)

# Water phase
def L_water_rule(model,i):
    return model.x[i] == pe.exp(-0.66037 - 7.1130*(539.1/model.T) - 0.67885*(1-model.T/539.1)**(1/3) -1.43381*(1-model.T/539.1))
model.L_water_con = pe.Constraint(model.COMP_WATER,rule=L_water_rule)

def V_water_rule(model,i):
    return model.y[i]*model.P == pe.exp(5.11564 - 1687.537/(model.T+230.17-273.15))
model.V_water_con = pe.Constraint(model.COMP_WATER,rule=V_water_rule)

In [9]:
# add bounds specifically for water
model.x['H2O'].setub(water_x[1]+abs(water_x[1])*0.2)
model.x['H2O'].setlb(water_x[0]-abs(water_x[0])*0.2)

model.y['H2O'].setub(water_yp[1]/model.P.lb)
model.y['H2O'].setlb(water_yp[0]/model.P.ub)

## Summation

\begin{equation}
\sum_ix_{i} = 1 \\
\sum_iy_{i} = 1
\end{equation}

In [10]:
def summation_x_main_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == 1
model.summation_x_main_con = pe.Constraint(rule=summation_x_main_rule)

def summation_y_main_rule(model):
    return sum(model.y[i] for i in m.COMP_TOTAL) == 1
model.summation_y_main_con = pe.Constraint(rule=summation_y_main_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [11]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) - model.W*model.energy_block.dH_L['H2O'] == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [12]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


## Load a sample vapor feed from reactive flash example

In [13]:
with open('../saved_solutions/reactive_flash_300C.pickle', 'rb') as f:
    C300 = pickle.load(f)

## Fixing In/Product Redundent Flow

In [14]:
model.x_.fix(0)
for i in m.COMP_TOTAL:
    model.y_['in',i].fix(C300.Solution.Variable['y[{}]'.format(i)]['Value'])
model.L['in'].fix(0)
model.L['P'].fix(0)
model.V['in'].fix(1)
model.V['P'].fix(0)
model.H_L_.fix(0)
model.H_V_.fix(C300.Solution.Variable['H_V']['Value'])

## Fixing Model Parameters

In [15]:
model.P.fix(21)
model.T_F.fix(200+273.15)
model.F.fix(0)
model.z['CO'].fix(0.3333)
model.z['H2'].fix(0.6666)
model.z['C30H62'].fix(0.0001)
model.VLE_block.n_ave.fix(58)

In [16]:
check_DOF(pe,model)

Active Equality Constraints:	 727
Active Inequality Constraints:	 0
Active Variables:		 899
Fixed Variables:		 171
DOF:				 1


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [17]:
model.T.setub(40+273.15)

In [18]:
# model.obj = pe.Objective(expr = model.L['out'],sense=pe.maximize)
# model.obj = pe.Objective(expr = model.Q_main,sense=pe.maximize)
model.obj = pe.Objective(expr = model.T,sense=pe.maximize)

In [19]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma97'
opt.options['nlp_scaling_method'] = None
opt.options['bound_relax_factor'] = 0
# opt.options['bound_push'] = 1e-30
# opt.options['bound_frac'] = 1e-30

opt.options['halt_on_ampl_error'] = 'yes'
opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma97
nlp_scaling_method=None
bound_relax_factor=0
halt_on_ampl_error=yes
max_iter=7000


List of user-set options:

                                    Name   Value                used
                      bound_relax_factor = 0                     yes
                           linear_solver = ma97                  yes
                                max_iter = 7000                  yes
                      nlp_scaling_method = None                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, r

In [20]:
model.Q_main.value

-35.547767013693104

In [21]:
model.T.value

313.1499981550555

In [22]:
model.W.value

0.5209583897318576

In [23]:
# model.solutions.store_to(results)
# with open('../saved_solutions/condenser_40C.pickle','wb') as f:
#     pickle.dump(results,f)

In [24]:
print('Component\t\tLiquid: {:.2f}\t\tVapor: {:.2f}'.format(model.L['out'].value,model.V['out'].value))
print('-'*72)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:.2%}\t\t\t{:.2%}'.format(model.x[i].value,model.y[i].value))

Component		Liquid: 0.09		Vapor: 0.39
------------------------------------------------------------------------
H2         		0.41%			9.16%
CO         		0.06%			0.76%
CO2        		12.14%			19.87%
H2O        		0.00%			1.54%
C2H4       		13.16%			11.35%
C3H6       		17.86%			3.99%
C4H8       		11.23%			1.01%
C5H10      		5.26%			0.15%
C6H12      		2.57%			0.02%
C7H14      		1.18%			0.00%
C8H16      		0.52%			0.00%
C9H18      		0.23%			0.00%
C10H20     		0.10%			0.00%
C11H22     		0.03%			0.00%
C12H24     		0.01%			0.00%
C13H26     		0.00%			0.00%
C14H28     		0.00%			0.00%
C15H30     		0.00%			0.00%
C16H32     		0.00%			0.00%
C17H34     		0.00%			0.00%
C18H36     		0.00%			0.00%
C19H38     		0.00%			0.00%
C20H40     		0.00%			0.00%
C1H4       		11.44%			45.33%
C2H6       		9.51%			5.52%
C3H8       		4.71%			0.94%
C4H10      		3.71%			0.28%
C5H12      		3.13%			0.07%
C6H14      		1.51%			0.01%
C7H16      		0.69%			0.00%
C8H18      		0.30%			0.00%
C9H20      		0.13%			0.00%
C10H22     		0.06%

In [25]:
model.VLE_block.pprint()

VLE_block : Size=1, Index=None, Active=True
    5 Set Declarations
        COMP_HENRY : Dim=0, Dimen=1, Size=9, Domain=None, Ordered=False, Bounds=None
            ['C1H4', 'C2H4', 'C2H6', 'C3H6', 'C3H8', 'CO', 'CO2', 'H2', 'H2O']
        COMP_NONHENRY : Dim=0, Dimen=1, Size=70, Domain=None, Ordered=False, Bounds=None
            Virtual
        COMP_NONHENRY_index_0 : Dim=0, Dimen=1, Size=70, Domain=None, Ordered=False, Bounds=None
            Virtual
        COMP_WATER : Dim=0, Dimen=1, Size=1, Domain=None, Ordered=False, Bounds=None
            ['H2O']
        f_V_con_index : Dim=0, Dimen=1, Size=79, Domain=None, Ordered=False, Bounds=None
            Virtual

    11 Var Declarations
        Hen : Size=9, Index=VLE_block.COMP_HENRY
            Key  : Lower              : Value              : Upper              : Fixed : Stale : Domain
            C1H4 :  62.19805852532296 :  83.20893959535485 :  99.85072906682174 : False : False : NonNegativeReals
            C2H4 : 11.2484647968644

            Key     : Lower : Body                                                                                                                           : Upper : Active
             C10H20 :   0.0 :           -2.72709 + log( VLE_block.P_sat[C10H20] ) - 8.718595*VLE_block.P_sat_dY_inf + 0.2175393206832472*VLE_block.P_sat_dY0 :   0.0 :   True
             C10H22 :   0.0 :           -2.72709 + log( VLE_block.P_sat[C10H22] ) - 8.873769*VLE_block.P_sat_dY_inf + 0.2151069188297999*VLE_block.P_sat_dY0 :   0.0 :   True
             C11H22 :   0.0 :          -2.72709 + log( VLE_block.P_sat[C11H22] ) - 9.718595*VLE_block.P_sat_dY_inf + 0.20272332706887297*VLE_block.P_sat_dY0 :   0.0 :   True
             C11H24 :   0.0 :          -2.72709 + log( VLE_block.P_sat[C11H24] ) - 9.873769*VLE_block.P_sat_dY_inf + 0.20059394118239604*VLE_block.P_sat_dY0 :   0.0 :   True
             C12H24 :   0.0 :         -2.72709 + log( VLE_block.P_sat[C12H24] ) - 10.718595*VLE_block.P_sat_dY_inf + 0.18969649729

            Key     : Lower : Body                                                                            : Upper : Active
             C10H20 :   0.0 :     f_L[C10H20] - VLE_block.gamma[C10H20] * VLE_block.P_sat[C10H20] * x[C10H20] :   0.0 :   True
             C10H22 :   0.0 :     f_L[C10H22] - VLE_block.gamma[C10H22] * VLE_block.P_sat[C10H22] * x[C10H22] :   0.0 :   True
             C11H22 :   0.0 :     f_L[C11H22] - VLE_block.gamma[C11H22] * VLE_block.P_sat[C11H22] * x[C11H22] :   0.0 :   True
             C11H24 :   0.0 :     f_L[C11H24] - VLE_block.gamma[C11H24] * VLE_block.P_sat[C11H24] * x[C11H24] :   0.0 :   True
             C12H24 :   0.0 :     f_L[C12H24] - VLE_block.gamma[C12H24] * VLE_block.P_sat[C12H24] * x[C12H24] :   0.0 :   True
             C12H26 :   0.0 :     f_L[C12H26] - VLE_block.gamma[C12H26] * VLE_block.P_sat[C12H26] * x[C12H26] :   0.0 :   True
             C13H26 :   0.0 :     f_L[C13H26] - VLE_block.gamma[C13H26] * VLE_block.P_sat[C13H26] * x[C13H26] :

            Key     : Lower : Body                          : Upper : Active
             C10H20 :   0.0 :   P * y[C10H20] - f_V[C10H20] :   0.0 :   True
             C10H22 :   0.0 :   P * y[C10H22] - f_V[C10H22] :   0.0 :   True
             C11H22 :   0.0 :   P * y[C11H22] - f_V[C11H22] :   0.0 :   True
             C11H24 :   0.0 :   P * y[C11H24] - f_V[C11H24] :   0.0 :   True
             C12H24 :   0.0 :   P * y[C12H24] - f_V[C12H24] :   0.0 :   True
             C12H26 :   0.0 :   P * y[C12H26] - f_V[C12H26] :   0.0 :   True
             C13H26 :   0.0 :   P * y[C13H26] - f_V[C13H26] :   0.0 :   True
             C13H28 :   0.0 :   P * y[C13H28] - f_V[C13H28] :   0.0 :   True
             C14H28 :   0.0 :   P * y[C14H28] - f_V[C14H28] :   0.0 :   True
             C14H30 :   0.0 :   P * y[C14H30] - f_V[C14H30] :   0.0 :   True
             C15H30 :   0.0 :   P * y[C15H30] - f_V[C15H30] :   0.0 :   True
             C15H32 :   0.0 :   P * y[C15H32] - f_V[C15H32] :   0.0 :   True

            Key  : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         